In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scvi
import scanpy as sc
import matplotlib
import csv
import os
seed=10
os.chdir(".../Atlas/objects")
sc.logging.print_versions()

Global seed set to 0


-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                 9.2.0
absl                NA
asttokens           NA
attr                22.1.0
backcall            0.2.0
bottleneck          1.3.5
cffi                1.15.1
chex                0.1.5
colorama            0.4.5
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.3
decorator           4.4.2
defusedxml          0.7.1
deprecate           0.3.2
docrep              0.3.2
entrypoints         0.4
etils               0.8.0
executing           1.0.0
flax                0.6.0
fsspec              2022.8.2
google              NA
h5py                3.7.0
igraph              0.9.11
ipykernel           6.15.3
ipython_genutils    0.2.0
ipywidgets          8.0.2
jax                 0.3.17
jaxlib              0.3.15
jedi                0.18.0
joblib              1.2.0
jupyter_server      2.0.4
kiwisolver          1.4.4
leidenalg           0.8.10
llvmlite            0.39.1
louvain          

In [ ]:
adata = sc.read("Atlas6.4_3000HVG_integrated_V2.h5ad")
adata

In [ ]:
model = scvi.model.SCVI.load(".../Atlas/Models/Model_Atlas6.1_V2", adata=adata, use_gpu=False)

In [7]:

clusters = ["DCT_CNT_CD"]

sc.tl.leiden(adata, resolution=1, restrict_to=("annotation_final_level1", clusters), random_state=0, key_added='leiden_restrict_DCT_CNT_CD')





In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.umap(adata, color = "leiden_restrict_DCT_CNT_CD")
sc.set_figure_params(figsize=(10,10))
sc.pl.umap(adata, color = "leiden_restrict_DCT_CNT_CD", legend_loc = "on data")

In [ ]:
#set clustering of interest
adata.obs['leiden'] = adata.obs['leiden_restrict_DCT_CNT_CD']
#get marker genes
de_df = model.differential_expression(
    groupby="leiden")
de_df.head()
markers = {}
cats = adata.obs.leiden.cat.categories
for i, c in enumerate(cats):
    cid = "{} vs Rest".format(c)
    leiden_scVI_df = de_df.loc[de_df.comparison == cid]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df.lfc_mean > 0]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["bayes_factor"] > 3]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["non_zeros_proportion1"] > 0.1]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["lfc_mean"] > 0.75]
    markers[c] = leiden_scVI_df.index.tolist()[:3]
    sc.tl.dendrogram(adata, groupby="leiden", use_rep="X_scVI")
    sc.pl.dotplot(
    adata,
    markers,
    groupby='leiden',
    dendrogram=True,
    color_map="Blues",
    swap_axes=True,
    use_raw=True,
    standard_scale="var",
)

In [ ]:
#set clustering of interest
adata.obs['leiden'] = adata.obs['leiden_restrict_TAL_MD']
#get marker genes
de_df = model.differential_expression(
    groupby="leiden")
de_df.head()
markers = {}
cats = adata.obs.leiden.cat.categories
for i, c in enumerate(cats):
    cid = "{} vs Rest".format(c)
    leiden_scVI_df = de_df.loc[de_df.comparison == cid]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df.lfc_mean > 0]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["bayes_factor"] > 3]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["non_zeros_proportion1"] > 0.1]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["lfc_mean"] > 0.75]
    markers[c] = leiden_scVI_df.index.tolist()[:3]
    sc.tl.dendrogram(adata, groupby="leiden", use_rep="X_scVI")
    sc.pl.dotplot(
    adata,
    markers,
    groupby='leiden',
    dendrogram=True,
    color_map="Blues",
    swap_axes=True,
    use_raw=True,
    standard_scale="var",
)

In [ ]:
adata_stromal = adata[adata.obs.annotation_final_level1 == "Stromal"]

sc.tl.dendrogram(adata_stromal, groupby="annotation_Atlas_level4", use_rep="X_scVI")


In [ ]:
sc.pl.dendrogram(adata_stromal, groupby = "annotation_Atlas_level4")

In [ ]:
adata_EC = adata[adata.obs.annotation_final_level1 == "EC"]

sc.tl.dendrogram(adata_EC, groupby="annotation_Atlas_level4", use_rep="X_scVI")

sc.pl.dendrogram(adata_EC, groupby = "annotation_Atlas_level4")

In [ ]:
adata_immune = adata[adata.obs.annotation_final_level1 == "Immune"]

sc.tl.dendrogram(adata_immune, groupby="annotation_Atlas_level4", use_rep="X_scVI")

sc.pl.dendrogram(adata_immune, groupby = "annotation_Atlas_level4")

In [ ]:
adata.obs["annotation_final_level2"] = adata.obs["annotation_Atlas_level4"]

Rename_cluster = {
    'CTAL': "TAL", 'CD_PC': "CD_PC", 'DCT_CNT': "DCT2", 'PTS1': "PTS1", 'CD_Pap': "CD_Pap", 
    'PTS2': "PTS2", 'ICA': "ICA", 'EC2': "EC_Peritub", 'DCT': "DCT", 'Str8': "Fib2", 
    'PTS3': "PTS3", 'ICB': "ICB", 'CNT': "CNT", 'Myeloid_1': "Myeloid", 'PEC': "PEC", 
    'EC1': "EC_Peritub", 'injPT': "injPT", 'DTL': "DTL", 'MD': "MD", 'Str1': "Fib1", 'EC4': "EC_DVR", 
    'Str5': "VSMC", 'Str9': "MC1", 'ATL': "ATL", 'Str3': "Fib1", 'ICA2': "ICA", 'Podo': "Podo", 
    'MTAL': "TAL", 'Str6': "Fib1", 'EC5': "EC_glom", 'Myeloid_2': "Myeloid", 'prolif_PT': "prolif_Tubule", 'B': "B", 
    'Str2': "Fib1", 'Lymph_1': "Lymphocyte", 'EC6': "EC_Lymph", 'Myeloid_5': "Myeloid", 'Myeloid_prolif': "Myeloid_prolif", 
    'EC3': "EC_Peritub", 'Str7': "Fib1", 'Str10': "Myofib", 'Myeloid_3': "Myeloid", 'Str4': "Fib1", 'Myeloid_4': "Myeloid", 
    'EC7': "EC_prolif", 'Str11': "VSMC", 'Myeloid_6': "Myeloid", 'Lymph_prolif': "Lymph_prolif", 'Lymph_2': "Lymphocyte", 
    'Plasma': "Plasma", 'Str13': "Neural", 'Str12': "Fib1"
}
adata.obs['annotation_final_level2'] = adata.obs['annotation_final_level2'].replace(Rename_cluster).astype('category')



In [ ]:
#load subclustered TAL 
adata_TAL = sc.read("Atlas6.3_subset_TAL_3000HVG_integratedV2.h5ad")

In [9]:
#we will add TAL1 and injured TAL in the next step based on subclustering and reintegration from TAL_only

Rename_cluster = {
    'TAL': "TAL2", 
}
adata.obs['annotation_final_level2'] = adata.obs['annotation_final_level2'].replace(Rename_cluster).astype('category')



In [ ]:
adata.obs["transfer"] = adata_TAL.obs['annotation_TAL']

# Create a mask indicating which cells are TAL-of-interest
mask = adata.obs['transfer'] == "TAL1"

# Add "MD" to the list of categories for the annotation column
adata.obs['annotation_final_level2'].cat.add_categories("TAL1", inplace=True)

# Update the annotation for cells with leiden resolution 2 equal to "30" to "MD"
adata.obs.loc[mask, 'annotation_final_level2'] = "TAL1"

In [ ]:
# Create a mask indicating which cells are TAL-of-interest
mask = adata.obs['transfer'] == "injTAL"

# Add "MD" to the list of categories for the annotation column
adata.obs['annotation_final_level2'].cat.add_categories("injTAL", inplace=True)

# Update the annotation for cells with leiden resolution 2 equal to "30" to "MD"
adata.obs.loc[mask, 'annotation_final_level2'] = "injTAL"

In [13]:
adata_DCT = sc.read("Atlas6.3_subset_DCT_CNT_CD_3000HVG_integratedV2.h5ad")

In [ ]:
adata.obs["transfer"] = adata_DCT.obs['leiden_scVI_0_5']

# Create a mask indicating which cells are TAL-of-interest
mask = adata.obs['transfer'] == "4"

# Add "MD" to the list of categories for the annotation column
adata.obs['annotation_final_level2'].cat.add_categories("injDCT_CNT", inplace=True)

# Update the annotation for cells with leiden resolution 2 equal to "30" to "MD"
adata.obs.loc[mask, 'annotation_final_level2'] = "injDCT_CNT"

In [ ]:
adata.obs["transfer"] = adata_TAL.obs['annotation_TAL']
sc.pl.umap(adata, color = "transfer", groups = "TAL1")

In [21]:

Rename_cluster = {
    'ATL': "DTL_ATL", 
    'DTL': "DTL_ATL", 
}
adata.obs['annotation_final_level2'] = adata.obs['annotation_final_level2'].replace(Rename_cluster).astype('category')



In [25]:
#export new metadata
df = adata.obs
df.to_csv(".../Atlas/Tables/Atlas_6.5_Metadata.csv")

In [ ]:
adata.write("Atlas6.5_3000HVG_integrated_V2.h5ad")

In [2]:
#create raw file based on Atlas 6.5
adata = sc.read("Atlas6.5_3000HVG_integrated_V2.h5ad")

adat = sc.read("Atlas6.0.h5ad")

In [ ]:
subset_list=adata.obs_names.tolist()
adat=adat[adat.obs_names.isin(subset_list),:] 
adat

In [ ]:
adat.obs = adata.obs
adat

In [5]:
adat.write("Atlas6.5_raw.h5ad")